# 소프트맥스 회귀의 비용 함수 구현

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

## 1. low-level로 구현

In [3]:
z = torch.FloatTensor([1, 2, 3])

In [10]:
# 소프트맥스 회귀를 거친 후 값 확인
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [11]:
sum(hypothesis)

tensor(1.)

소프트맥스 회귀를 거친 후 0과 1사이의 값을 가지는 벡터로 변환되었다. 모든 원소의 합은 1이다.

In [12]:
z = torch.rand(3, 5, requires_grad=True)

In [15]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


3 x 5 임의로 선언하여 소프트맥스 회귀를 적용하였다. 이 때, dim은 소프트맥스 회귀를 적용하는 차원을 의미한다. 각 행을 하나의 샘플데이터로 보고 소프트맥스 회귀를 적용해야 하므로 두 번째 차원에 적용하였다.

In [36]:
y = torch.randint(5, (3, )).long()
print(y, y.type())

y_ = torch.randint(5, (3, ))
print(y_, y_.type())

y_ = torch.IntTensor([1, 2])
print(y_, y_.type())

y_ = torch.IntTensor([1, 2]).long()
print(y_, y_.type())

tensor([0, 3, 2]) torch.LongTensor
tensor([2, 3, 2]) torch.LongTensor
tensor([1, 2], dtype=torch.int32) torch.IntTensor
tensor([1, 2]) torch.LongTensor


5개의 클래스 중 임의로 3개의 레이블을 만든다. long() 함수는 텐서의 타입을 LongTensor로 변경하는 역할을 수행한다. randint() 함수의 출력은 이미 LongTensor 타입으로 바뀌는 것은 없다. 

In [63]:
y.unsqueeze(1)

tensor([[0],
        [3],
        [2]])

In [130]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])

`scatter(dim, index, value)` 순서로 두 번째 차원으로 연산을 수행하고 값 1을 넣겠다는 의미. index의 값은 두 번째 차원의 값을 의미하기 때문에 3 x 5행렬에서는 5보다 작은 0 ~ 4의 값을 가져야 한다.

In [131]:
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])


In [145]:
cost = ((y_one_hot * -torch.log(hypothesis)).sum(dim=1)).mean()
print(cost)

tensor(1.4779, grad_fn=<MeanBackward0>)


`-` 부호를 앞에 붙였을 때랑 뒤에 붙였을 때 grad_fn이 MeanBackward / NegBackward로 변경되는 것을 알 수 있다. 

## 2. high-level로 구현

softmax 결과에 로그를 씌울 때 softmax 의 출력값을 log의 입력값으로 전달

In [149]:
z = torch.rand(3, 5, requires_grad=True)
torch.log(F.softmax(z, dim=1))

tensor([[-1.4661, -1.3655, -2.1714, -1.7770, -1.4664],
        [-1.2339, -1.2855, -1.9580, -1.9562, -1.8985],
        [-1.6426, -1.2071, -2.0212, -1.4390, -1.9821]], grad_fn=<LogBackward0>)

위의 과정을 결합한 log_softmax() 가 있다

In [153]:
F.log_softmax(z, dim=1)

tensor([[-1.4661, -1.3655, -2.1714, -1.7770, -1.4664],
        [-1.2339, -1.2855, -1.9580, -1.9562, -1.8985],
        [-1.6426, -1.2071, -2.0212, -1.4390, -1.9821]],
       grad_fn=<LogSoftmaxBackward0>)

비용함수의 경우 아래의 식을 통해 계산하였다

In [155]:
((y_one_hot) * - F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.8145, grad_fn=<MeanBackward0>)

nll_loss()를 사용하면 더 간단하게 표현 가능하다

In [156]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.8145, grad_fn=<NllLossBackward0>)

nll 이란 Negative Log Likehood의 약자이다. 위에서 nll_loss는 log_softmax를 수행한 후 남은 수식을 수행한다. 이를 더 간단하게 표현 가능하다.

In [157]:
F.cross_entropy(z, y)

tensor(1.8145, grad_fn=<NllLossBackward0>)

cross_entropy는 비용 함수에 소프트맥스 함수까지 포함하고 있다.

In [166]:
print(((y_one_hot) * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean())

# F.softmax() + torch.log() = F.log_softmax()
print(((y_one_hot) * -F.log_softmax(z, dim=1)).sum(dim=1).mean())

# F.log_softmax() + F.nll_loss() = F.cross_entropy()
print(F.nll_loss(F.log_softmax(z, dim=1), y))
print(F.cross_entropy(z, y))

tensor(1.8145, grad_fn=<MeanBackward0>)
tensor(1.8145, grad_fn=<MeanBackward0>)
tensor(1.8145, grad_fn=<NllLossBackward0>)
tensor(1.8145, grad_fn=<NllLossBackward0>)
